### Create environment

In [1]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import gym_kilobots

import pygame
import sys
# Define four objects in a centered square
square_objects = [
    ((-0.25,  0.25), 0.0),   # top-left
    (( 0.25,  0.25), 0.0),   # top-right
    (( 0.25, -0.25), 0.0),   # bottom-right
    ((-0.25, -0.25), 0.0),   # bottom-left
]

# Define 1 object in the center of the environment
single_center_object = [
    ((0.0, 0.0), 0.0)        # center,
]

# Choose a fixed light position
light_pos = (-0.70, 0.1)

# Choose four explicit kilobot positions
kb_positions = [
        (-0.70, 0.10),
        (-0.75, 0.15),
        (-0.8, 0.10),
        (-0.85, 0.15),
    ]

# NOTE: if you don't do this, the environment will randomly place the objects
env = gym.make(
    'Kilobots-QuadAssembly-v0',
    render_mode='human',
    num_kilobots=len(kb_positions),
    object_config=single_center_object,
    light_position=light_pos,
    kilobot_positions=kb_positions
)

check_env(env,warn=True)

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
/opt/anaconda3/envs/swarm/lib/python3.13/site-packages/stable_baselines3/common/env_checker.py:462: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [ ]:
#vec_env = make_vec_env(env, n_envs = 1)

# Train the agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=300000)
model.save("swarm_ppo_test")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -2.68e+03 |
| time/              |           |
|    fps             | 473       |
|    iterations      | 1         |
|    time_elapsed    | 4         |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -2.7e+03     |
| time/                   |              |
|    fps                  | 438          |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0034381317 |
|    clip_fraction        | 0.0164       |
|    clip_range           | 0.2         

In [ ]:
model = PPO.load("swarm_ppo_test")

def controller(x):
      action,states = model.predict(x)
      return action

running = True
obs, info = env.reset()

t = 0

running = True
while running:

      env.render()

      x = obs
      action = controller(x)
      obs, reward, terminated, truncated, info = env.step(action)

      print('Reward: ', reward)

      # Print the orientation of the objects using the unwrapped environment
      #print(env.unwrapped.get_objects_status())

      # End episode
      if terminated or truncated:
            break
            running = False

      t += 1
      
env.close()  # Ensure the environment is properly closed
pygame.display.quit
pygame.quit()


Reward:  4.075530653340222e-07
Reward:  1.6044161638423198e-06
Reward:  2.9152398598229147e-07
Reward:  1.7634796981966624e-06
Reward:  5.864965384591265e-06
Reward:  1.8451528690397646e-06
Reward:  2.8994857168646126e-07
Reward:  1.5103074191184945e-06
Reward:  2.467399380113017e-07
Reward:  1.7317930212198771e-06
Reward:  2.8647361403586044e-07
Reward:  1.8873070664195584e-06
Reward:  6.885258290466428e-06
Reward:  9.283754935896821e-07
Reward:  2.780835638953532e-07
Reward:  1.5455303020572875e-06
Reward:  2.395412572629368e-07
Reward:  1.5103074191184945e-06
Reward:  4.0272475033583424e-07
Reward:  2.082960853293256e-06
Reward:  3.18424121687904e-07
Reward:  5.040390923855174e-08
Reward:  3.18424121687904e-07
Reward:  1.5103074191184945e-06
Reward:  2.5162876413081783e-07
Reward:  1.4066986801586496e-06
Reward:  2.35872821990862e-07
Reward:  1.1740255526527618e-06
Reward:  8.53950972287761e-06
Reward:  1.5705580481350625e-06
Reward:  3.129537506927111e-07
Reward:  2.110690800880396

: 